In [32]:
import math

def evaluate(predict_label_and_marked_label_list):
    """
    :param predict_label_and_marked_label_list: 一个元组列表。例如
    [ ([1, 2, 3, 4, 5], [4, 5, 6, 7]),
      ([3, 2, 1, 4, 7], [5, 7, 3])
     ]
    需要注意这里 predict_label 是去重复的，例如 [1,2,3,2,4,1,6]，去重后变成[1,2,3,4,6]
    :return:
    """
    right_label_num = 0
    right_label_at_pos_num = [0, 0, 0, 0, 0]
    sample_num = 0
    all_marked_label_num = 0
    for predict_labels, marked_labels in predict_label_and_marked_label_list:
        sample_num += 1
        marked_label_set = set(marked_labels)
        all_marked_label_num += len(marked_label_set)
        for pos, label in zip(range(0, min(len(predict_labels), 5)), predict_labels):
            if label in marked_label_set:
                right_label_num += 1
                right_label_at_pos_num[pos] = right_label_at_pos_num[pos] + 1

    precision = 0.0
    for pos, right_num in zip(range(0, 5), right_label_at_pos_num):
        precision += ((right_num / float(sample_num))) / math.log(2.0 + pos)
    recall = float(right_label_num) / all_marked_label_num

    return (precision * recall) / (precision + recall), precision, recall, right_label_num, all_marked_label_num

In [1]:
import pandas as pd
import numpy as np

In [2]:
%%time
questions = pd.read_csv('./question_train_word.csv')
questions['topics'] = questions.topics.apply(lambda s: s.split(','))

CPU times: user 20.6 s, sys: 5.4 s, total: 26 s
Wall time: 28.8 s


In [3]:
topics = [t for topics in questions.topics for t in topics]
set_topics = set(topics)
print(len(topics))
print(len(set_topics))

7014424
1999


In [4]:
%%time
questions['title'] = questions.title.astype('U').apply(lambda s: s.split(','))
print(questions.iloc[9])

Unnamed: 0                                                      9
Unnamed: 0.1                                                    9
id                                            3174606710238304130
title           [w2218, w54, w1038, w125529, w90, w7665, w6, w...
description     w85,w54,w86,w54,w87,w54,w2008,w18335,w1417,w54...
topics                                     [-4115748438709160582]
Name: 9, dtype: object
CPU times: user 8.58 s, sys: 2.56 s, total: 11.1 s
Wall time: 12.8 s


In [5]:
topics = pd.read_csv('./topic_info.txt', sep='\t', names=['id', 'p_ids', 'c', 'w', 'd_c', 'd_w'])
topics.iloc[0]

id                                      738845194850773558
p_ids                                 -5833678375673307423
c                                                    c0,c1
w                                                       w0
d_c      c0,c1,c2,c3,c4,c5,c6,c7,c0,c1,c8,c9,c10,c11,c1...
d_w      w0,w1,w2,w3,w4,w5,w6,w7,w8,w9,w10,w11,w12,w13,...
Name: 0, dtype: object

In [6]:
topics.set_index('id', inplace=True)
topics.iloc[0]

p_ids                                 -5833678375673307423
c                                                    c0,c1
w                                                       w0
d_c      c0,c1,c2,c3,c4,c5,c6,c7,c0,c1,c8,c9,c10,c11,c1...
d_w      w0,w1,w2,w3,w4,w5,w6,w7,w8,w9,w10,w11,w12,w13,...
Name: 738845194850773558, dtype: object

In [7]:
topics['ws'] = topics['w'].apply(lambda x: x.split(','))

In [8]:
questions.iloc[0]

Unnamed: 0                                                      0
Unnamed: 0.1                                                    0
id                                            6555699376639805223
title           [w305, w13549, w22752, w11, w7225, w2565, w110...
description     w231,w54,w1681,w54,w11506,w5714,w7,w54,w744,w1...
topics                 [7739004195693774975, 3738968195649774859]
Name: 0, dtype: object

In [9]:
from tqdm import tqdm
%pdb
result = []
for i, q in questions[:100].iterrows():
    
    for t in q[5]:
        topic_w = topics.loc[int(t)]
        for w in topic_w.ws:
            if w not in q[3]:
                result.append(False)
                break
            result.append(True)

Automatic pdb calling has been turned ON


In [10]:
print(len(result))
print(len([i for i in result if i]))

242
84


In [11]:
topic_word_map = dict(zip(topics.w, topics.index.astype('U')))
topic_word_map['w0']

'738845194850773558'

In [12]:
from collections import Counter
_topics = [t for topics in questions.topics for t in topics]
_c = Counter(_topics)
_c.most_common(3)

[('7476760589625268543', 65989),
 ('4697014490911193675', 49192),
 ('-4653836020042332281', 45444)]

In [58]:
most_common_topic = '7476760589625268543'

set_topics_w = [w for ws in topics.ws for w in ws]
def predict(title):
    intersection = set(title).intersection(set_topics_w)
    topic_ids = []
    if intersection:
        topic_ids = [i for i in map(lambda x: topic_word_map.get(x), intersection) if i]
#     else:
#         topic_ids = [most_common_topic]
    return topic_ids

print(questions.iloc[3])
print(predict(questions.iloc[3].title))

Unnamed: 0                                                      3
Unnamed: 0.1                                                    3
id                                              -5698296155734268
title           [w8646, w2744, w1462, w9, w54, w138, w54, w50,...
description                                                   NaN
topics                [-6758942141122113907, 3195914392210930723]
Name: 3, dtype: object
['3195914392210930723']


In [59]:
y_predict = []
for i, q in questions[:10000].iterrows():
    y_predict.append(predict(q[3]))

In [60]:
%pdb
evaluate(r_m_result)

Automatic pdb calling has been turned OFF


(0.1518606400531184, 0.6395453756981615, 0.19914867740954698, 5240, 26312)

In [15]:
count = 0
for i in range(len(y_predict)):
    for t in y_predict[i]:
        if t in questions.iloc[i].topics:
            count +=1
print(count)

548


In [16]:
print(y_predict[:10])
print(y_predict[3])
print(set(questions.iloc[3].title).intersection(set_topics_w))
print(questions.iloc[3].topics)
print(list(map(lambda x: topics.loc[int(x)].w, questions.iloc[3].topics)))

[[], ['-3026760652624195547'], ['-8252369929269451135'], ['3195914392210930723'], ['4195795391451929480'], [], ['-5932391056759866388', '-5839832565699655443'], ['42427966967759255'], ['1158968214639900190', '-9165709055872875620'], ['-4115748438709160582']]
['3195914392210930723']
{'w138', 'w6', 'w54'}
['-6758942141122113907', '3195914392210930723']
['w8652,w54,w674,w8647,w614', 'w138']


In [53]:
r_m_result = list(zip(y_predict, questions[:10000].topics))
r_m_result[0]

([], ['7739004195693774975', '3738968195649774859'])

In [42]:
%%time
colnames = ['id', 'c', 'w', 'd_c', 'd_w']
test_questions = pd.read_csv('./question_eval_set.txt', sep='\t', names=colnames)

CPU times: user 3.08 s, sys: 371 ms, total: 3.45 s
Wall time: 3.61 s


In [44]:
test_ids = set(test_questions['id'])
train_ids = set(questions['id'])

In [45]:
test_ids.intersection(train_ids)

set()

In [47]:
test_questions['title'] = test_questions.w.astype('U').apply(lambda s: s.split(','))

In [49]:
test_questions.iloc[0]

id                                     6215603645409872328
c        c924,c531,c102,c284,c188,c104,c98,c107,c11,c11...
w        w1340,w1341,w55,w1344,w58,w6,w24178,w26959,w47...
d_c      c1128,c529,c636,c572,c1321,c139,c540,c223,c510...
d_w      w4094,w1618,w20104,w19234,w1097,w1005,w4228,w2...
title    [w1340, w1341, w55, w1344, w58, w6, w24178, w2...
Name: 0, dtype: object

In [62]:
%%time
y_test = []
for i, q in test_questions.iterrows():
    y_test.append(predict(q[5]))

CPU times: user 29.1 s, sys: 205 ms, total: 29.3 s
Wall time: 29.7 s


In [63]:
y_test[:10]

[[],
 ['2858911571784840089', '-240041917918953337'],
 [],
 [],
 ['-8963554618409314978'],
 ['-202658488388760612'],
 ['4482402820945758152'],
 [],
 ['8690965822342756180',
  '-7653703019053330516',
  '-268886646698338239',
  '738845194850773558'],
 ['3195914392210930723']]

In [64]:
test_questions['predict'] = y_test

In [69]:
test_questions['predict'] = test_questions.predict.apply(lambda x: ','.join(x))
test_questions.iloc[1]

id                                       6649324930261961840
c          c346,c1549,c413,c294,c675,c504,c183,c74,c541,c...
w          w40132,w1357,w1556,w1380,w2464,w33,w16791,w109...
d_c                                                      NaN
d_w                                                      NaN
title      [w40132, w1357, w1556, w1380, w2464, w33, w167...
predict              2858911571784840089,-240041917918953337
Name: 1, dtype: object

In [91]:
%pdb
import csv
test_questions['result'] = test_questions[['id', 'predict']].apply(lambda x: ','.join(x), axis=1)

Automatic pdb calling has been turned OFF


In [98]:
test_questions['result'].to_csv('baseline_tag_in_text.csv', header=False, index=False, sep=' ')

In [96]:
help(test_questions['result'].to_csv)

Help on method to_csv in module pandas.core.series:

to_csv(path=None, index=True, sep=',', na_rep='', float_format=None, header=False, index_label=None, mode='w', encoding=None, date_format=None, decimal='.') method of pandas.core.series.Series instance
    Write Series to a comma-separated values (csv) file
    
    Parameters
    ----------
    path : string or file handle, default None
        File path or object, if None is provided the result is returned as
        a string.
    na_rep : string, default ''
        Missing data representation
    float_format : string, default None
        Format string for floating point numbers
    header : boolean, default False
        Write out series name
    index : boolean, default True
        Write row names (index)
    index_label : string or sequence, default None
        Column label for index column(s) if desired. If None is given, and
        `header` and `index` are True, then the index names are used. A
        sequence should be 

In [80]:
test_questions.iloc[1].id

6649324930261961840

## Score: 0.128724958515191